# DSCI 417 – Homework 04
**Lauren Forti**

In [0]:
# setup
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, expr

spark = SparkSession.builder.getOrCreate()

## Load Diamond Data

In [0]:
# create schema
my_schema = 'carat DOUBLE, cut STRING, color STRING, clarity STRING, depth DOUBLE, table DOUBLE, price INTEGER, x DOUBLE, y DOUBLE, z DOUBLE'

# read file into df
diamonds = (
    spark.read
    .option('delimiter', '\t')
    .option('header', True)
    .schema(my_schema)
    .csv('/FileStore/tables/diamonds.txt')
)

# display schema
diamonds.printSchema()

root
-- carat: double (nullable = true)
-- cut: string (nullable = true)
-- color: string (nullable = true)
-- clarity: string (nullable = true)
-- depth: double (nullable = true)
-- table: double (nullable = true)
-- price: integer (nullable = true)
-- x: double (nullable = true)
-- y: double (nullable = true)
-- z: double (nullable = true)

## Part 1:  Grouping By Cut

In [0]:
# function to convert str to numeric
def rank_cut(level):
  ranks = {
    'Fair':1,
    'Good':2,
    'Very Good':3,
    'Premium':4,
    'Ideal':5
  }
  # return appropriate level
  return ranks.get(level)

# register as Spark UDF
spark.udf.register('rank_cut', rank_cut)

Out[3]: <function __main__.rank_cut(level)>

In [0]:
(
  diamonds
  
  # group by cut
  .groupBy('cut')
  
  .agg(
    # count diaamonds
    expr('count(*) as n_diamonds'),
    # calc avg price
    expr('int(round(avg(price),0)) as avg_price'),
    # calc avg carat
    expr('round(avg(carat), 2) as avg_carat'),
    # calc avg depth
    expr('round(avg(depth), 2) as avg_depth'),
    # calc avg table
    expr('round(avg(table), 2) as avg_table'),
  )

  # convert cut column to numeric
  .withColumn('cut', expr('rank_cut(cut)'))
  
  # sort by cut
  .sort('cut')

).show()

+---+----------+---------+---------+---------+---------+
cut|n_diamonds|avg_price|avg_carat|avg_depth|avg_table|
+---+----------+---------+---------+---------+---------+
 1| 1610| 4359| 1.05| 64.04| 59.05|
 2| 4906| 3929| 0.85| 62.37| 58.69|
 3| 12082| 3982| 0.81| 61.82| 57.96|
 4| 13791| 4584| 0.89| 61.26| 58.75|
 5| 21551| 3458| 0.7| 61.71| 55.95|
+---+----------+---------+---------+---------+---------+

## Problem 2: Filtering based on Carat Size

In [0]:
# get count of diamonds within ranges
for item in range(0,6):
  # get bounds
  lower = item
  upper = item + 1
  
  # count diamonds within bounds
  ct = diamonds.filter((col('carat') >= lower) & (col('carat') < upper)).count()
  
  # output results
  print(f'The number of diamonds with carat size in range [{lower}, {upper}) is {ct}.')

The number of diamonds with carat size in range [0, 1) is 34880.
The number of diamonds with carat size in range [1, 2) is 16906.
The number of diamonds with carat size in range [2, 3) is 2114.
The number of diamonds with carat size in range [3, 4) is 34.
The number of diamonds with carat size in range [4, 5) is 5.
The number of diamonds with carat size in range [5, 6) is 1.

## Problem 3: Binning by Carat Size

In [0]:
# function to find which bin diamonds fall into
def carat_bin(size):
  bins = ['[0, 1)', '[1, 2)', '[2, 3)', '[3, 4)', '[4, 5)', '[5, 6)']
  return bins[int(size)]

# register as UDF
spark.udf.register('carat_bin', carat_bin)

Out[6]: <function __main__.carat_bin(size)>

In [0]:
(
  diamonds
  
  # add new calcd column
  .select(
    '*',
    expr('carat_bin(carat) as carat_bin')
  )
  
  # group by carat_bin
  .groupBy('carat_bin')
  
  .agg(
    # get count of diamonds
    expr('count(*) as n_diamonds'),
    # calc avg price
    expr('int(round(avg(price),0)) as avg_price')
  )
  
  # sort by carat_bin asc
  .sort('carat_bin')

).show()

+---------+----------+---------+
carat_bin|n_diamonds|avg_price|
+---------+----------+---------+
 [0, 1)| 34880| 1633|
 [1, 2)| 16906| 7288|
 [2, 3)| 2114| 14847|
 [3, 4)| 34| 14309|
 [4, 5)| 5| 16458|
 [5, 6)| 1| 18018|
+---------+----------+---------+

## Load IMDB Data

In [0]:
# read file into df
movies = (
    spark.read
    .option('delimiter', '\t')
    .option('header', True)
    .csv('/FileStore/tables/imdb/movies.txt')
)

# display schema
movies.printSchema()

root
-- imdb_title_id: string (nullable = true)
-- title: string (nullable = true)
-- year: string (nullable = true)
-- genre: string (nullable = true)
-- duration: string (nullable = true)
-- country: string (nullable = true)
-- language: string (nullable = true)

In [0]:
# read file into df
names = (
    spark.read
    .option('delimiter', '\t')
    .option('header', True)
    .csv('/FileStore/tables/imdb/names.txt')
)

# display schema
names.printSchema()

root
-- imdb_name_id: string (nullable = true)
-- name: string (nullable = true)
-- birth_name: string (nullable = true)
-- height: string (nullable = true)
-- bio: string (nullable = true)
-- date_of_birth: string (nullable = true)
-- date_of_death: string (nullable = true)

In [0]:
# read file into df
title_principals = (
    spark.read
    .option('delimiter', '\t')
    .option('header', True)
    .csv('/FileStore/tables/imdb/title_principals.txt')
)

# display schema
title_principals.printSchema()

root
-- imdb_title_id: string (nullable = true)
-- ordering: string (nullable = true)
-- imdb_name_id: string (nullable = true)
-- category: string (nullable = true)
-- characters: string (nullable = true)

In [0]:
# read file into df
ratings = (
    spark.read
    .option('delimiter', '\t')
    .option('header', True)
    .csv('/FileStore/tables/imdb/ratings.txt')
)

# display schema
ratings.printSchema()

root
-- imdb_title_id: string (nullable = true)
-- rating: string (nullable = true)
-- total_votes: string (nullable = true)

In [0]:
# output # of records in each of the four dfs
print('   Number of Records')
print('-'*24)
print('movies           ', movies.count())
print('names            ', names.count())
print('title_principals ', title_principals.count())
print('ratings          ', ratings.count())

Number of Records
------------------------
movies 85855
names 297710
title_principals 835513
ratings 85855

## Problem 4: Number of Appearances by Actor

In [0]:
(
  title_principals
  
  # get only actors/actresses
  .filter((col('category') == 'actor') | (col('category') == 'actress'))
  
  # group by imdb ID for each person
  .groupBy('imdb_name_id')
  
  # get # of movies each person has appeared in
  .agg(
    expr('count(imdb_name_id) as appearances')
   )
  
  # combine with names df
  .join(other=names, on='imdb_name_id', how='left')
  
  # select only name and appearances cols
  .select('name', 'appearances')
  
  # sort desc by # of appearances
  .sort('appearances', ascending = False)
  
  # display the first 16 rows
  .show(16)
)

+-----------------+-----------+
 name|appearances|
+-----------------+-----------+
 Mohanlal| 163|
 Amitabh Bachchan| 142|
 Mammootty| 140|
 Eric Roberts| 133|
 John Wayne| 132|
 Gérard Depardieu| 130|
 Prakash Raj| 125|
 Akshay Kumar| 115|
 Michael Madsen| 107|
 Andy Lau| 102|
Catherine Deneuve| 101|
 Brahmanandam| 99|
 Anupam Kher| 99|
 Michael Caine| 94|
 Ajay Devgn| 94|
 Christopher Lee| 93|
+-----------------+-----------+
only showing top 16 rows

## Problem 5: Average Rating by Director

In [0]:
(
  title_principals
  
  # get only actors/actresses
  .filter(col('category') == 'director')
  
  # bring in ratings info
  .join(other=ratings, on='imdb_title_id', how='outer')
  
  # group by imdb ID for each person
  .groupBy('imdb_name_id')
  
  .agg(
    # get # of movies each person has directed
    expr('count(imdb_name_id) as num_films'),
    # get total votes for each director
    expr('int(sum(total_votes)) as total_votes'),
    # get avg rating for each director
    expr('round(avg(rating),2) as avg_rating')
  )
  
  # keep only directors whose movies have @ least 1 million votes  
  .filter(
    expr('total_votes >= 1000000')
  )

  # keep only directors who have directed at least 5 movies
  .filter(
    expr('num_films >= 5')
  )
  
  # bring in names df
  .join(other=names, on='imdb_name_id', how='left')
  
  # select cols
  .select('name', 'num_films', 'total_votes', 'avg_rating')
  
  # sort desc by # of appearances
  .sort('avg_rating', ascending = False)
  
  # display the first 16 rows
  .show(16, truncate=False)
)

+---------------------+---------+-----------+----------+
name |num_films|total_votes|avg_rating|
+---------------------+---------+-----------+----------+
Christopher Nolan |11 |11653144 |8.22 |
Lee Unkrich |5 |3329612 |8.14 |
Hayao Miyazaki |12 |2254496 |8.01 |
Sergio Leone |7 |1720654 |7.93 |
Quentin Tarantino |14 |9460772 |7.93 |
Stanley Kubrick |13 |4232356 |7.78 |
David Fincher |10 |6944421 |7.76 |
Sam Mendes |10 |3067512 |7.73 |
Wes Anderson |9 |2173090 |7.61 |
Alejandro G. Iñárritu|7 |2067540 |7.61 |
Peter Jackson |13 |7304418 |7.58 |
Brad Bird |6 |2294748 |7.57 |
Alfonso Cuarón |8 |2078975 |7.54 |
Andrew Stanton |5 |2649551 |7.52 |
Bong Joon Ho |8 |1172684 |7.51 |
Akira Kurosawa |32 |1061519 |7.51 |
+---------------------+---------+-----------+----------+
only showing top 16 rows

## Problem 6: Actors Appearing in Horror Films

In [0]:
# filter df for horror movies
horror_films = movies.filter(expr('genre LIKE "%Horror%"'))

# display # of records
print(horror_films.count())

9557

In [0]:
(
  title_principals
  
  # get only actors/actresses
  .filter((col('category') == 'actor') | (col('category') == 'actress'))
  
  # filter join with horror_films df
  .join(other=horror_films, on='imdb_title_id', how='semi')
  
  # group by imdb ID for each person
  .groupBy('imdb_name_id')
  
  .agg(
    # get # of movies each person has been in
    expr('count(imdb_name_id) as num_films'),    
  )

  # bring in names df
  .join(other=names, on='imdb_name_id', how='left')
  
  # select cols
  .select('name', 'num_films')
  
  # sort desc by # of appearances
  .sort('num_films', ascending = False)

  # display the first 16 rows
  .show(16)
)

+----------------+---------+
 name|num_films|
+----------------+---------+
 Christopher Lee| 56|
 Peter Cushing| 47|
 Boris Karloff| 46|
 John Carradine| 43|
 Bela Lugosi| 38|
 Vincent Price| 34|
 Lance Henriksen| 33|
 Eric Roberts| 29|
 Lon Chaney Jr.| 28|
 Bill Moseley| 27|
 Tony Todd| 27|
Donald Pleasence| 26|
 Paul Naschy| 26|
 Robert Englund| 23|
 Brad Dourif| 23|
 Sergey A.| 23|
+----------------+---------+
only showing top 16 rows